#### Q&A Application with GraphDB

In [24]:
NEO4J_URI = "neo4j+s://a97abe53.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "g1dBHw6-VAlpxy2FPUwTgbeuSbMmwKmqCzWSiLVjePw"

In [25]:
import os

os.environ["NEO4J_URI"] = "neo4j+s://a97abe53.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "g1dBHw6-VAlpxy2FPUwTgbeuSbMmwKmqCzWSiLVjePw"

In [40]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, refresh_schema=False)
graph

In [69]:
## Load the movie related dataset

movie_query = """
    LOAD CSV WITH HEADERS FROM
    'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row
    MERGE(m:Movie{id:row.movieId})\
    SET m.released = date(row.released),
        m.title = row.title,
        m.imdbRating = toFloat(row.imdbRating)

    FOREACH (director in split(row.director, '|') |
        MERGE (p:Person {name:trim(director)})
        MERGE (p)-[:DIRECTED]->(m))
    FOREACH (actors in split(row.actors, '|') |
        MERGE (p:Person {name:trim(actors)})
        MERGE (p)-[:ACTED_IN]->(m))
    FOREACH (genre in split(row.genre, '|') |
        MERGE (g:Genre {name:trim(genre)})
        MERGE (g)-[:IN_GENRE]->(m))
        
    RETURN m.id AS movieId, m.title AS title, m.released AS released, m.imdbRating AS imdbRating
    ORDER BY m.id
"""

movie_query

"\n    LOAD CSV WITH HEADERS FROM\n    'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n    MERGE(m:Movie{id:row.movieId})    SET m.released = date(row.released),\n        m.title = row.title,\n        m.imdbRating = toFloat(row.imdbRating)\n\n    FOREACH (director in split(row.director, '|') |\n        MERGE (p:Person {name:trim(director)})\n        MERGE (p)-[:DIRECTED]->(m))\n    FOREACH (actors in split(row.actors, '|') |\n        MERGE (p:Person {name:trim(actors)})\n        MERGE (p)-[:ACTED_IN]->(m))\n    FOREACH (genre in split(row.genre, '|') |\n        MERGE (g:Genre {name:trim(genre)})\n        MERGE (g)-[:IN_GENRE]->(m))\n        \n    RETURN m.id AS movieId, m.title AS title, m.released AS released, m.imdbRating AS imdbRating\n    ORDER BY m.id\n"

In [71]:
# Assuming `graph` is your Neo4jGraph instance
def print_graph_schema(graph):
    # Print node properties
    print("Node properties:")
    for label, props in graph.get_node_properties().items():
        print(f"{label} {props}")

    # Print relationship properties
    print("\nRelationship properties:")
    rel_props = graph.get_relationship_properties()
    if not rel_props:
        print("(none)")
    else:
        for rel_type, props in rel_props.items():
            print(f"{rel_type} {props}")

    # Print relationships
    print("\nThe relationships:")
    for rel in graph.get_relationships():
        print(rel)

# Call the function
print_graph_schema(graph)


Node properties:


AttributeError: 'Neo4jGraph' object has no attribute 'get_node_properties'

In [64]:
# Run this to see the data in your database
check_query = "MATCH (m:Movie) RETURN m.title, m.imdbRating LIMIT 5"

with graph._driver.session(database=graph._database) as session:
    result = session.run(check_query)
    for record in result:
        print(record.data())

{'m.title': 'Toy Story', 'm.imdbRating': 8.3}
{'m.title': 'Jumanji', 'm.imdbRating': 6.9}
{'m.title': 'Grumpier Old Men', 'm.imdbRating': 6.6}
{'m.title': 'Waiting to Exhale', 'm.imdbRating': 5.6}
{'m.title': 'Father of the Bride Part II', 'm.imdbRating': 5.9}
